# 부록 10.2.2: 도구 사용으로 JSON 강제하기

## 학습 목표

* 구조화된 응답을 강제하기 위해 도구를 사용하는 방법 이해하기
* 이 "트릭"을 활용하여 구조화된 JSON 생성하기

Claude를 활용하는 더 흥미로운 방법 중 하나는 JSON과 같은 구조화된 콘텐츠로 응답하도록 강제하는 것입니다. 엔티티 추출, 데이터 요약, 감정 분석 등 Claude에게 표준화된 JSON 응답을 받고 싶은 상황이 많습니다.

이를 위한 한 가지 방법은 단순히 Claude에게 JSON으로 응답하라고 요청하는 것이지만, 이렇게 하면 Claude로부터 받은 큰 문자열에서 실제로 JSON을 추출하거나 원하는 정확한 형식의 JSON을 따르도록 하는 추가 작업이 필요할 수 있습니다.

다행히도 **Claude가 도구를 사용하고 싶을 때마다 우리가 도구를 정의할 때 지정한 완벽하게 구조화된 형식으로 응답합니다.**

이전 수업에서 Claude에게 계산기 도구를 제공했습니다. 도구를 사용하고 싶을 때 다음과 같은 콘텐츠로 응답했습니다.

```
{
    'operand1': 1984135, 
    'operand2': 9343116, 
    'operation': 'multiply'
}
```

이것은 JSON과 매우 유사해 보입니다!

Claude가 구조화된 JSON을 생성하게 하고 싶다면 이 점을 활용할 수 있습니다. 해야 할 일은 특정 JSON 구조를 설명하는 도구를 정의하고 Claude에게 알려주는 것뿐입니다. 그러면 Claude는 "도구를 호출"한다고 생각하며 응답할 것이지만, 사실 우리가 원하는 것은 구조화된 응답일 뿐입니다.

***

# 개념 개요

이것이 이전 수업에서 했던 것과 어떻게 다른지 살펴보겠습니다. 다음은 이전 수업의 워크플로우 다이어그램입니다.

![chickens_calculator.png](./images/chickens_calculator.png)

이전 수업에서는 Claude에게 도구를 제공했고, Claude가 도구를 호출하고 싶어 했으며, 실제로 기본 도구 함수를 호출했습니다.

이번 수업에서는 Claude를 "속이겠습니다". 특정 도구에 대해 알려주지만 실제로 기본 도구 함수를 호출할 필요는 없습니다. 특정 응답 구조를 강제하기 위해 도구를 사용하는 것입니다. 다음 다이어그램을 참조하세요.

![structured_response.png](./images/structured_response.png)

## 감정 분석
간단한 예시로 시작해 보겠습니다. Claude에게 텍스트의 감정을 분석하고 다음과 같은 형태의 JSON 객체로 응답하게 하고 싶다고 가정해 봅시다.

```
{
  "negative_score": 0.6,
  "neutral_score": 0.3,
  "positive_score": 0.1
}
```

해야 할 일은 이 형태를 JSON 스키마를 사용하여 도구로 정의하는 것뿐입니다. 다음은 잠재적인 구현 예시입니다.

In [ ]:
%pip install -qU pip
%pip install -qUr requirements.txt

In [ ]:
import boto3
import json
from datetime import datetime
from botocore.exceptions import ClientError

# Import the hints module from the utils package
from utils import hints

session = boto3.Session()
region = session.region_name

In [ ]:
modelId = 'anthropic.claude-3-sonnet-20240229-v1:0'
#modelId = 'anthropic.claude-3-haiku-20240307-v1:0'

bedrock_client = boto3.client(service_name = 'bedrock-runtime', region_name = region,)

In [ ]:
tools = {
  "tools": [
    {
      "toolSpec": {
        "name": "print_sentiment_scores",
        "description": "Prints the sentiment scores of a given text.",
        "inputSchema": {
          "json": {
            "type": "object",
            "properties": {
              "positive_score": {
                "type": "number",
                "description": "The positive sentiment score, ranging from 0.0 to 1.0."},
              "negative_score": {
                "type": "number",
                "description": "The negative sentiment score, ranging from 0.0 to 1.0."},
              "neutral_score": {
                "type": "number",
                "description": "The neutral sentiment score, ranging from 0.0 to 1.0."}
            },
            "required": ["positive_score", "negative_score", "neutral_score"
            ]
          }
        }
      }
    }
  ]
}

이제 우리는 Claude에게 이 도구에 대해 알려주고 Claude에게 이 도구를 사용하라고 명시적으로 지시할 수 있습니다. 그러면 Claude가 실제로 그 도구를 사용할 것입니다. Claude가 도구를 사용하겠다는 응답을 받아야 합니다. 도구 사용 응답에는 우리가 원하는 정확한 형식의 모든 데이터가 포함되어야 합니다.

In [ ]:
tweet = "I'm a HUGE hater of pickles.  I actually despise pickles.  They are garbage."

query = f"""
<text>
{tweet}
</text>

Only use the print_sentiment_scores tool.
"""

converse_api_params = {
    "modelId": "anthropic.claude-3-haiku-20240307-v1:0",
    "messages": [{"role": "user", "content": [{"text": query}]}],
    "inferenceConfig": {"temperature": 0.0, "maxTokens": 400},
    "toolConfig": tools,
}

response = bedrock_client.converse(**converse_api_params)

In [ ]:
response['output']

Claude로부터 받은 응답을 살펴봅시다. 중요한 부분을 굵게 표시했습니다:

>{'message': {'role': 'assistant',
  'content': [{'text': '주어진 텍스트에 대한 감정 분석 결과입니다:'},
   {'toolUse': {'toolUseId': 'tooluse_d2ReNcjDQvKjLLet4u9EOA',
     'name': 'print_sentiment_scores',
     **'input': {'positive_score': 0.0,
      'negative_score': 0.7,
      'neutral_score': 0.3}**}}]}}

Claude는 이 감정 분석 데이터를 사용할 도구를 호출한다고 "생각"하지만, 실제로는 우리가 그 데이터를 추출하여 JSON으로 변환할 것입니다.

In [ ]:
import json
json_sentiment = None
for content in response['output']['message']['content']:
    if isinstance(content, dict) and 'toolUse' in content:
        tool_use = content['toolUse']
        if tool_use['name'] == "print_sentiment_scores":
            json_sentiment = tool_use['input']
            break

if json_sentiment:
    print("Sentiment Analysis (JSON):")
    print(json.dumps(json_sentiment, indent=2))
else:
    print("No sentiment analysis found in the response.")

잘 되었습니다! 이제 트윗이나 기사를 입력받아 감정 분석 결과를 JSON으로 출력하거나 반환하는 재사용 가능한 함수를 만들어 봅시다.

In [ ]:
def analyze_sentiment(content):

    query = f"""
    <text>
    {content}
    </text>

    Only use the print_sentiment_scores tool.
    """

    converse_api_params = {
        "modelId": "anthropic.claude-3-haiku-20240307-v1:0",
        "messages": [{"role": "user", "content": [{"text": query}]}],
        "inferenceConfig": {"temperature": 0.0, "maxTokens": 4096},
        "toolConfig": tools,
    }

    response = bedrock_client.converse(**converse_api_params)

    json_sentiment = None
    for content in response['output']['message']['content']:
        if isinstance(content, dict) and 'toolUse' in content:
            tool_use = content['toolUse']
            if tool_use['name'] == "print_sentiment_scores":
                json_sentiment = tool_use['input']
                break

    if json_sentiment:
        print("Sentiment Analysis (JSON):")
        print(json.dumps(json_sentiment, indent=2))
    else:
        print("No sentiment analysis found in the response.")

In [ ]:
analyze_sentiment("OMG I absolutely love taking bubble baths soooo much!!!!")

In [ ]:
analyze_sentiment("Honestly I have no opinion on taking baths")

***

## `toolChoice`를 사용하여 도구 사용 강제하기

현재 우리는 프롬프트를 통해 Claude에게 `print_sentiment_scores` 도구를 사용하도록 "강제"하고 있습니다. 프롬프트에서 `print_sentiment_scores 도구만 사용하세요.`라고 쓰면 대개 작동하지만, 더 나은 방법이 있습니다! `tool_choice` 매개변수를 사용하여 Claude에게 특정 도구를 사용하도록 강제할 수 있습니다.

```json
tool_choice = {
    "tool": {
        "name": "print_sentiment_scores"}
}
```

위의 코드는 Claude에게 `print_sentiment_scores` 도구를 호출하여 응답해야 한다고 알려줍니다. 이제 도구와 함수를 업데이트해 봅시다.

In [ ]:
# create out toolConfig var and force the toolChoice to be print_sentiment_scores by name
toolConfig = {'tools': [],
        "toolChoice": {
        "tool": {"name":"print_sentiment_scores"},
    }
}

In [ ]:
# append our tool specification to our toolConfig
toolConfig['tools'].append({
      "toolSpec": {
        "name": "print_sentiment_scores",
        "description": "Prints the sentiment scores of a given text.",
        "inputSchema": {
          "json": {
            "type": "object",
            "properties": {
              "positive_score": {
                "type": "number",
                "description": "The positive sentiment score, ranging from 0.0 to 1.0."},
              "negative_score": {
                "type": "number",
                "description": "The negative sentiment score, ranging from 0.0 to 1.0."},
              "neutral_score": {
                "type": "number",
                "description": "The neutral sentiment score, ranging from 0.0 to 1.0."}
            },
            "required": ["positive_score", "negative_score", "neutral_score"]
          }
        }
      }
    })

In [ ]:
# optional uncomment if you want to see the complete toolConfig
toolConfig

In [ ]:
def analyze_sentiment(content):

    query = f"""
    <text>
    {content}
    </text>

    Only use the print_sentiment_scores tool.
    """

    converse_api_params = {
        "modelId": "anthropic.claude-3-haiku-20240307-v1:0",
        "messages": [{"role": "user", "content": [{"text": query}]}],
        "inferenceConfig": {"temperature": 0.0, "maxTokens": 4096},
        "toolConfig": toolConfig
    }

    response = bedrock_client.converse(**converse_api_params)

    json_sentiment = None
    for content in response['output']['message']['content']:
        if isinstance(content, dict) and 'toolUse' in content:
            tool_use = content['toolUse']
            if tool_use['name'] == "print_sentiment_scores":
                json_sentiment = tool_use['input']
                break

    if json_sentiment:
        print("Sentiment Analysis (JSON):")
        print(json.dumps(json_sentiment, indent=2))
    else:
        print("No sentiment analysis found in the response.")

In [ ]:
analyze_sentiment("Honestly I have no opinion on taking baths")

앞으로 있을 수업에서 `toolChoice`에 대해 더 자세히 다룰 것입니다.

***

## 엔티티 추출 예시

이번에는 같은 방식으로 Claude에게 텍스트 샘플에서 사람, 조직, 위치 등의 엔티티를 추출하여 잘 정리된 JSON 형식으로 응답하도록 해봅시다.


In [ ]:
toolConfig = {
  "tools": [
    {
      "toolSpec": {
        "name": "print_entities",
        "description": "Prints extract named entities.",
        "inputSchema": {
          "json": {
            "type": "object",
            "properties": {
              "entities": {
                "type": "array",
                "items": {
                  "type": "object",
                  "properties": {
                    "name": {"type": "string", "description": "The extracted entity name."},
                    "type": {"type": "string", "description": "The entity type (e.g., PERSON, ORGANIZATION, LOCATION)."},
                    "context": {"type": "string", "description": "The context in which the entity appears in the text."}
                  },
                  "required": ["name", "type", "context"]
                }
              }
            },
            "required": ["entities"]
          }
        }
      }
    }
  ]
}

text = "John works at Google in New York. He met with Sarah, the CEO of Acme Inc., last week in San Francisco."

query = f"""
<document>
{text}
</document>

Use the print_entities tool.
"""

converse_api_params = {
    "modelId": "anthropic.claude-3-haiku-20240307-v1:0",
    "messages": [{"role": "user", "content": [{"text": query}]}],
    "additionalModelRequestFields": {"max_tokens": 4096},
    "toolConfig": toolConfig
}

response = bedrock_client.converse(**converse_api_params)


json_entities = None
for content in response['output']['message']['content']:
    if isinstance(content, dict) and 'toolUse' in content:
        tool_use = content['toolUse']
        if tool_use['name'] == "print_entities":
            json_entities = tool_use['input']
            break

if json_entities:
    print("Extracted Entities (JSON):")
    print(json.dumps(json_entities, indent=2))
else:
    print("No entities found in the response.")

앞서와 마찬가지로 같은 "트릭"을 사용합니다. Claude에게 특정 도구에 접근할 수 있다고 알려서 Claude가 특정 데이터 형식으로 응답하도록 합니다. 그런 다음 Claude가 응답한 형식화된 데이터를 추출하면 됩니다.

이 사용 사례에서는 Claude에게 특정 도구를 사용하라고 명시적으로 지시하는 것이 도움이 됩니다.


>print_entities 도구를 사용하세요.


***

## 더 복잡한 데이터를 사용한 Wikipedia 요약 예시

좀 더 복잡한 예시를 해봅시다. Python `wikipedia` 패키지를 사용하여 전체 Wikipedia 문서를 가져와 Claude에게 전달합니다. Claude에게 다음과 같은 내용을 포함하는 응답을 생성하도록 합니다.

* 문서의 주요 주제
* 문서의 요약
* 문서에서 언급된 키워드 및 주제 목록
* 문서의 카테고리 분류 목록(엔터테인먼트, 정치, 비즈니스 등) 및 분류 점수(해당 주제가 그 카테고리에 속하는 정도)

우리가 클로드에게 월트 디즈니에 대한 위키백과 기사를 전달하면, 다음과 같은 결과를 예상할 수 있습니다: 

```json
{
  "subject": "Walt Disney",
  "summary": "Walter Elias Disney는 미국의 애니메이터, 영화 제작자, 기업가였습니다. 그는 미국 애니메이션 산업의 선구자였고 만화 제작에 여러 가지 발전을 이끌었습니다. 그는 개인으로는 가장 많은 아카데미상을 수상하고 후보에 올랐습니다. 또한 디즈니랜드와 다른 테마파크, 그리고 TV 프로그램 개발에도 관여했습니다.",
  "keywords": [
    "Walt Disney",
    "animation",
    "film producer",
    "entrepreneur",
    "Disneyland",
    "theme parks",
    "television"
  ],
  "categories": [
    {
      "name": "Entertainment",
      "score": 0.9
    },
    {
      "name": "Business",
      "score": 0.7
    },
    {
      "name": "Technology",
      "score": 0.6
    }
  ]
}
```

위키백과 페이지 주제를 입력받아 해당 기사를 찾고, 내용을 다운로드한 다음 클로드에 전달하여 결과 JSON 데이터를 출력하는 함수의 구현 예시입니다. 클로드의 응답 형식을 "코치"하기 위해 도구를 정의하는 동일한 전략을 사용합니다.

참고: 머신에 wikipedia가 설치되어 있지 않다면 `pip install wikipedia`를 실행하세요!

In [ ]:
import wikipedia

#tool definition
toolConfig = {
  "tools": [
    {
      "toolSpec": {
        "name": "print_article_classification",
        "description": "Prints the classification results.",
        "inputSchema": {
          "json": {
            "type": "object",
            "properties": {
              "subject": {
                "type": "string",
                "description": "The overall subject of the article"},
              "summary": {
                "type": "string",
                "description": "A paragaph summary of the article"},
              "keywords": {
                "type": "array",
                "items": {
                  "type": "string",
                  "description": "List of keywords and topics in the article"}
              },
              "categories": {
                "type": "array",
                "items": {
                  "type": "object",
                  "properties": {
                    "name": {"type": "string", "description": "The category name."},
                    "score": {"type": "number", "description": "The classification score for the category, ranging from 0.0 to 1.0."}
                  },
                  "required": ["name", "score"]
                }
              }
            },
            "required": ["subject", "summary", "keywords", "categories"]
          }
        }
      }
    }
  ]
}

#The function that generates the json for a given article subject
def generate_json_for_article(subject):
    page = wikipedia.page(subject, auto_suggest=True)
    query = f"""
    <document>
    {page.content}
    </document>

    Use the print_article_classification tool. Example categories are Politics, Sports, Technology, Entertainment, Business.
    """

    converse_api_params = {
        "modelId": "anthropic.claude-3-haiku-20240307-v1:0",
        "messages": [{"role": "user", "content": [{"text": query}]}],
        "additionalModelRequestFields": {"max_tokens": 4096},
        "toolConfig": toolConfig
    }

    response = bedrock_client.converse(**converse_api_params)

    json_classification = None
    for content in response['output']['message']['content']:
        if isinstance(content, dict) and 'toolUse' in content:
            tool_use = content['toolUse']
            if tool_use['name'] == "print_article_classification":
                json_classification = tool_use['input']
                break

    if json_classification:
        print("Text Classification (JSON):")
        print(json.dumps(json_classification, indent=2))
    else:
        print("No text classification found in the response.")

In [ ]:
generate_json_for_article("Jeff Goldblum")

In [ ]:
generate_json_for_article("Octopus")

In [ ]:
generate_json_for_article("Herbert Hoover")

***

## 연습

위의 전략을 사용하여 단어나 구를 입력받아 영어 원문과 스페인어, 프랑스어, 일본어, 아랍어로 번역된 구조화된 JSON 출력을 생성하는 `translate` 함수를 작성하세요.

다음과 같이 작동해야 합니다:

이렇게 호출하면:

In [ ]:
translate("how much does this cost")

**1단계.** "translations_from_claude"라는 이름의 도구에 대한 toolSpec을 포함하는 toolConfig를 완성해야 합니다. 여기에는 영어가 포함된 toolSpec의 시작 부분이 있습니다.

In [ ]:
# Here is a strater toolConfig we've added "English" for you.
toolConfig = {
  "tools": [
    {
      "toolSpec": {
        "name": "translations_from_claude",
        "description": "The translations from Claude of a user provided phrase into English to Spanish, French, Japanese, and Arabic.",
        "inputSchema": {
          "json": {
            "type": "object",
            "properties": {
              "english": {"type": "string", "description": "Your English translation of the provided content from the user"},
            },
            "required": ["english"]
          }
        }
      }
    }
  ],
    "toolChoice": {"tool": {"name": "translations_from_claude"}}
}

❓ toolSpec과 관련된 힌트가 필요하면 아래 셀을 실행하세요!

In [ ]:
print(hints.exercise_10_2_2_toolSpec)

**2단계.** 이제 "translate" 함수를 완성해야 합니다.

In [ ]:
def translate(query):
    prompt = f"""
    Translate phrase below from the user into Spanish, French, Japanese and Arabic.
    Content to translate: '{query}'
    """

    pass

❓ translate 함수와 관련된 힌트가 필요하면 아래 셀을 실행하세요!

In [ ]:
print(hints.exercise_10_2_2_translate)

**3단계.** 이제 translate 함수를 실행해 봅시다.

In [ ]:
translate("how much does this cost")

다음과 같은 출력을 예상합니다: 

```json
{
  "english": "how much does this cost",
  "spanish": "¿cuánto cuesta esto?",
  "french": "combien ça coûte?",
  "japanese": "これはいくらですか",
  "arabic": "كم تكلفة هذا؟"
}
```

**참고: 결과를 출력하려면 다음 코드 라인이 도움이 될 것입니다:**

In [ ]:
print(json.dumps(translations_from_claude, ensure_ascii=False, indent=2))